In [1]:
import pandas as pd

In [54]:
Lab = pd.read_csv("FullData/LabReduced.csv", usecols = ['Patient_ID', 'PerformedDate', 'Name_calc', 'TestResult_calc', 'UnitOfMeasure_calc'])
Exam = pd.read_csv("FullData/Exam.csv", usecols = ['Patient_ID', 'Exam1', 'Result1_calc', 'Exam2', 'Result2_calc', 'UnitOfMeasure_calc', 'DateCreated'])


In [55]:
Exam1 = Exam.drop(columns=['Exam2','Result2_calc']).rename(columns={'Exam1':'Name_calc','Result1_calc':'TestResult_calc'})
Exam2 = Exam.drop(columns=['Exam1','Result1_calc']).rename(columns={'Exam2':'Name_calc','Result2_calc':'TestResult_calc'})
Exam = pd.concat([Exam1,Exam2]).dropna()
print(Exam['Name_calc'].value_counts(ascending=False))

dBP (mmHg)                  2570192
sBP (mmHg)                  2570192
Weight (kg)                 1102589
BMI (kg/m^2)                 910459
Height (cm)                  736942
Waist Circumference (cm)     115353
PEFR (L/min)                     35
Name: Name_calc, dtype: int64


In [56]:
tests = ['Alanine Aminotransferase in Serum or Plasma (ALT)',
              'Aspartate Aminotransferase in Serum or Plasma (AST)',
              'Urea (mM)',
              'Plasma Creatinine Clearance (eGFR)',
              'Potassium (mM)',
              'Sodium (mM)',
              'Gamma Glutamyl Transferase in Serum or Plasma (GGT)',
              'Leukocytes in Blood (WBC)',
              'Protein, total (g/L)',
              'Total Cholesterol (TCh)',
              'Total Bilirubin (TBIL)',
              'Platelets in Blood (PLT)',
              'Fasting Blood Glucose (FBG)',
              'Hemoglobin (Hb)',
              'Erythrocytes in Blood (RBC)',
              'Albumin (g/L)',
              'Hemoglobin A1c (HbA1c)',
              'BMI (kg/m^2)']
Lab = Lab.loc[Lab.Name_calc.isin(tests)]
Exam = Exam.loc[Exam.Name_calc.isin(tests)]

In [57]:
Exam = Exam.rename(columns={'DateCreated':'PerformedDate'})
Exam = Exam[Exam.columns[[0,4,1,2,3]]]

In [99]:
Exam = Exam.drop_duplicates(subset=['Patient_ID','PerformedDate'])

In [100]:
lab = pd.concat([Lab,Exam])

In [101]:
lab.isna().sum()

Patient_ID                 0
PerformedDate         432244
Name_calc                  0
TestResult_calc            0
UnitOfMeasure_calc     28971
dtype: int64

In [102]:
lab = lab.dropna(subset = ['PerformedDate'])
lab.isna().sum()

Patient_ID                0
PerformedDate             0
Name_calc                 0
TestResult_calc           0
UnitOfMeasure_calc    28660
dtype: int64

In [103]:
len(lab.index)

11930362

In [104]:
disease_periods = pd.read_csv('AP_dp.csv', index_col=None)
disease_periods = disease_periods.drop(disease_periods.columns[0], axis=1)

In [105]:
disease_periods.dtypes

Patient_ID      int64
CompOnset      object
DateOfOnset    object
dtype: object

In [106]:
disease_periods.head()

,Patient_ID,CompOnset,DateOfOnset
0,4000000526483,1992-10-05,1991-01-22
1,4000000528945,1996-05-09,1994-07-29
2,4000000527165,1998-10-08,1994-05-16
3,4000000531095,1999-06-28,1998-11-04
4,4000000610981,1999-07-05,1991-03-25


In [107]:
patient_records = lab.loc[lab['Patient_ID'].isin(disease_periods['Patient_ID'])]

In [108]:
print(len(patient_records.index),patient_records.columns)

349738 Index(['Patient_ID', 'PerformedDate', 'Name_calc', 'TestResult_calc',
       'UnitOfMeasure_calc'],
      dtype='object')


In [109]:
patient_records = pd.merge(patient_records, disease_periods, on='Patient_ID')

In [110]:
print(len(patient_records.index),patient_records.columns)

349738 Index(['Patient_ID', 'PerformedDate', 'Name_calc', 'TestResult_calc',
       'UnitOfMeasure_calc', 'CompOnset', 'DateOfOnset'],
      dtype='object')


In [111]:
patient_records.head()

,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,CompOnset,DateOfOnset
0,1000100164719,2013-10-11,Platelets in Blood (PLT),225,10^9/L,2019-07-25,2013-03-26
1,1000100164719,2013-09-22,Platelets in Blood (PLT),243,10^9/L,2019-07-25,2013-03-26
2,1000100164719,2016-02-22,Leukocytes in Blood (WBC),6.2,10^9/L,2019-07-25,2013-03-26
3,1000100164719,2016-09-12,Hemoglobin A1c (HbA1c),7.1,%,2019-07-25,2013-03-26
4,1000100164719,2013-10-11,Erythrocytes in Blood (RBC),5,10^12/L,2019-07-25,2013-03-26


In [112]:
relVisits = patient_records[(patient_records['PerformedDate'] < patient_records['CompOnset']) & (patient_records['PerformedDate'] >= patient_records['DateOfOnset'])]


In [113]:
len(relVisits.index)

123404

In [114]:
relVisits.PerformedDate = pd.to_datetime(relVisits.PerformedDate,format='%Y-%m-%d')
relVisits.CompOnset = pd.to_datetime(relVisits.CompOnset,format='%Y-%m-%d')
relVisits.DateOfOnset = pd.to_datetime(relVisits.DateOfOnset,format='%Y-%m-%d')

/Users/davidhuang/miniforge3/lib/python3.9/site-packages/pandas/core/generic.py:5489: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [115]:
relVisits.TestResult_calc = pd.to_numeric(relVisits.TestResult_calc,errors='coerce')
relVisits = relVisits.dropna()

In [116]:
print(relVisits.isna().sum())
print(relVisits.dtypes)

Patient_ID            0
PerformedDate         0
Name_calc             0
TestResult_calc       0
UnitOfMeasure_calc    0
CompOnset             0
DateOfOnset           0
dtype: int64
Patient_ID                     int64
PerformedDate         datetime64[ns]
Name_calc                     object
TestResult_calc              float64
UnitOfMeasure_calc            object
CompOnset             datetime64[ns]
DateOfOnset           datetime64[ns]
dtype: object


In [117]:
#remove vast minority units (mg/L for Total Bilirubin (n = 40) and mmol/L for Hemoglobin (n = 10))
badIndices = relVisits.loc[(relVisits.Name_calc == 'Total Bilirubin (TBIL)') & (relVisits.UnitOfMeasure_calc == 'mg/L')].index
badIndices2 = relVisits.loc[(relVisits.Name_calc == 'Hemoglobin (Hb)') & (relVisits.UnitOfMeasure_calc == 'mmol/L')].index
relVisits = relVisits.drop(badIndices)
relVisits = relVisits.drop(badIndices2)

In [146]:
Results = relVisits

In [147]:
for test in tests:
    q1 = Results.loc[Results.Name_calc == test].TestResult_calc.quantile(.25)
    q3 = Results.loc[Results.Name_calc == test].TestResult_calc.quantile(.75)
    rng = q3 - q1
    badIndices = Results.loc[(Results.Name_calc == test) & ((Results.TestResult_calc < (q1 - rng)) 
                                    |(Results.TestResult_calc > (q3 + rng)))].index
    Results = Results.drop(badIndices)

In [148]:
Results.head()

,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,CompOnset,DateOfOnset
0,1000100164719,2013-10-11,Platelets in Blood (PLT),225.0,10^9/L,2019-07-25,2013-03-26
1,1000100164719,2013-09-22,Platelets in Blood (PLT),243.0,10^9/L,2019-07-25,2013-03-26
2,1000100164719,2016-02-22,Leukocytes in Blood (WBC),6.2,10^9/L,2019-07-25,2013-03-26
3,1000100164719,2016-09-12,Hemoglobin A1c (HbA1c),7.1,%,2019-07-25,2013-03-26
4,1000100164719,2013-10-11,Erythrocytes in Blood (RBC),5.0,10^12/L,2019-07-25,2013-03-26


In [149]:
len(Results.index)

113160

In [150]:
turtiles = pd.read_csv('TestTurtiles.csv')

In [151]:
turtiles

,Unnamed: 0,TestName,t1,t2
0,0,Alanine Aminotransferase in Serum or Plasma (ALT),18.00,27.00
1,1,Aspartate Aminotransferase in Serum or Plasma ...,18.00,24.00
2,2,Urea (mM),5.10,6.70
3,3,Plasma Creatinine Clearance (eGFR),60.00,83.00
4,4,Potassium (mM),4.20,4.60
5,5,Sodium (mM),139.00,141.00
6,6,Gamma Glutamyl Transferase in Serum or Plasma ...,23.00,41.00
7,7,Leukocytes in Blood (WBC),6.20,7.90
8,8,"Protein, total (g/L)",63.00,71.00
9,9,Total Cholesterol (TCh),3.53,4.40


In [158]:
for index, row in Results.iterrows() :
    test = row['Name_calc']
    value = row['TestResult_calc']
    feature = [0,0,0]
    t1 = float(turtiles.loc[turtiles['TestName'] == test]['t1'])
    t2 = float(turtiles.loc[turtiles['TestName'] == test]['t2'])
    if value < t1 :
        feature = 1.0 #[1,0,0]
    elif value > t2 :
        feature = 2.0 #[0,1,0]
    else :
        feature = 3.0 #[0,0,1]
    Results.at[index, 'TestResult_calc'] = feature
    
        

In [159]:
Results.head()

,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,CompOnset,DateOfOnset,225.0
0,1000100164719,2013-10-11,Platelets in Blood (PLT),3.0,10^9/L,2019-07-25,2013-03-26,NaN
1,1000100164719,2013-09-22,Platelets in Blood (PLT),3.0,10^9/L,2019-07-25,2013-03-26,NaN
2,1000100164719,2016-02-22,Leukocytes in Blood (WBC),3.0,10^9/L,2019-07-25,2013-03-26,NaN
3,1000100164719,2016-09-12,Hemoglobin A1c (HbA1c),3.0,%,2019-07-25,2013-03-26,NaN
4,1000100164719,2013-10-11,Erythrocytes in Blood (RBC),2.0,10^12/L,2019-07-25,2013-03-26,NaN


In [160]:
indices = []
for index, row in Results.iterrows() :
    temp = (str(row['Patient_ID']),str(row['PerformedDate']))
    indices.append(temp)

In [163]:
AllVisits = {}
VisitValues = {}
for index, row in Results.iterrows() :
    temp = (str(row['Patient_ID']),str(row['PerformedDate']))
    diag = (row['Name_calc'],row['TestResult_calc'])
    if temp not in AllVisits.keys():
        AllVisits[temp] = []
    AllVisits[temp] = AllVisits[temp] + [diag]

PVDF = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in AllVisits.items() ]))
PVDF = PVDF.transpose()
PVDF = PVDF.dropna(axis=0,how='all') # Remove NA


In [164]:
PVDF

0   \
1000100164719  2013-10-11 00:00:00            (Platelets in Blood (PLT), 3.0)   
               2013-09-22 00:00:00            (Platelets in Blood (PLT), 3.0)   
               2016-02-22 00:00:00           (Leukocytes in Blood (WBC), 3.0)   
               2016-09-12 00:00:00              (Hemoglobin A1c (HbA1c), 3.0)   
               2017-10-16 00:00:00  (Plasma Creatinine Clearance (eGFR), 2.0)   
...                                                                       ...   
5050010047249  2019-11-11 00:00:00                        (BMI (kg/m^2), 3.0)   
11114000017506 2006-07-25 00:00:00                        (BMI (kg/m^2), 1.0)   
               2006-02-21 00:00:00                        (BMI (kg/m^2), 1.0)   
               2008-03-25 00:00:00                        (BMI (kg/m^2), 1.0)   
11112099925406 2010-01-13 00:00:00                        (BMI (kg/m^2), 3.0)   

                                                                           1   \
1000100164719  2013-10-11 00:00:00         (Erythrocytes in Blood (RBC), 2.0)   
               2013-09-22 00:00:00         (Erythrocytes in Blood (RBC), 2.0)   
               2016-02-22 00:00:00  (Plasma Creatinine Clearance (eGFR), 2.0)   
               2016-09-12 00:00:00                                        NaN   
               2017-10-16 00:00:00              (Hemoglobin A1c (HbA1c), 2.0)   
...                                                                       ...   
5050010047249  2019-11-11 00:00:00                                        NaN   
11114000017506 2006-07-25 00:00:00                                        NaN   
               2006-02-21 00:00:00                                        NaN   
               2008-03-25 00:00:00                                        NaN   
11112099925406 2010-01-13 00:00:00                                        NaN   

                                                                  2   \
1000100164719  2013-10-11 00:00:00  (Leukocytes in Blood (WBC), 1.0)   
               2013-09-22 00:00:00  (Leukocytes in Blood (WBC), 1.0)   
               2016-02-22 00:00:00            (Hemoglobin (Hb), 2.0)   
               2016-09-12 00:00:00                               NaN   
               2017-10-16 00:00:00                               NaN   
...                                                              ...   
5050010047249  2019-11-11 00:00:00                               NaN   
11114000017506 2006-07-25 00:00:00                               NaN   
               2006-02-21 00:00:00                               NaN   
               2008-03-25 00:00:00                               NaN   
11112099925406 2010-01-13 00:00:00                               NaN   

                                                                                   3   \
1000100164719  2013-10-11 00:00:00  (Alanine Aminotransferase in Serum or Plasma (...   
               2013-09-22 00:00:00  (Alanine Aminotransferase in Serum or Plasma (...   
               2016-02-22 00:00:00                    (Platelets in Blood (PLT), 3.0)   
               2016-09-12 00:00:00                                                NaN   
               2017-10-16 00:00:00                                                NaN   
...                                                                               ...   
5050010047249  2019-11-11 00:00:00                                                NaN   
11114000017506 2006-07-25 00:00:00                                                NaN   
               2006-02-21 00:00:00                                                NaN   
               2008-03-25 00:00:00                                                NaN   
11112099925406 2010-01-13 00:00:00                                                NaN   

                                                                                   4   \
1000100164719  2013-10-11 00:00:00  (Gamma Glutamyl Transferase in Serum or Plasma...   
               20

In [168]:
eTests = []
for t in tests :
    for i in range(1,4) :
        eTests.append(t+str(i))

In [170]:
RowFeatures = pd.DataFrame(data = 0,
                       index = indices,
                       columns=eTests)
RowFeatures

,Alanine Aminotransferase in Serum or Plasma (ALT)1,Alanine Aminotransferase in Serum or Plasma (ALT)2,Alanine Aminotransferase in Serum or Plasma (ALT)3,Aspartate Aminotransferase in Serum or Plasma (AST)1,Aspartate Aminotransferase in Serum or Plasma (AST)2,Aspartate Aminotransferase in Serum or Plasma (AST)3,Urea (mM)1,Urea (mM)2,Urea (mM)3,Plasma Creatinine Clearance (eGFR)1,...,Erythrocytes in Blood (RBC)3,Albumin (g/L)1,Albumin (g/L)2,Albumin (g/L)3,Hemoglobin A1c (HbA1c)1,Hemoglobin A1c (HbA1c)2,Hemoglobin A1c (HbA1c)3,BMI (kg/m^2)1,BMI (kg/m^2)2,BMI (kg/m^2)3
"(1000100164719, 2013-10-11 00:00:00)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(1000100164719, 2013-09-22 00:00:00)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(1000100164719, 2016-02-22 00:00:00)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(1000100164719, 2016-09-12 00:00:00)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(1000100164719, 2013-10-11 00:00:00)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(5050010047249, 2019-11-11 00:00:00)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(11114000017506, 2006-07-25 00:00:00)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(11114000017506, 2006-02-21 00:00:00)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(11114000017506, 2008-03-25 00:00:00)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [171]:
for index, row in RowFeatures.iterrows() :
    for t in PVDF.loc[index].dropna() :
        test = t[0] + str(int(t[1]))
        row[test] = 1

In [172]:
RowFeatures

,Alanine Aminotransferase in Serum or Plasma (ALT)1,Alanine Aminotransferase in Serum or Plasma (ALT)2,Alanine Aminotransferase in Serum or Plasma (ALT)3,Aspartate Aminotransferase in Serum or Plasma (AST)1,Aspartate Aminotransferase in Serum or Plasma (AST)2,Aspartate Aminotransferase in Serum or Plasma (AST)3,Urea (mM)1,Urea (mM)2,Urea (mM)3,Plasma Creatinine Clearance (eGFR)1,...,Erythrocytes in Blood (RBC)3,Albumin (g/L)1,Albumin (g/L)2,Albumin (g/L)3,Hemoglobin A1c (HbA1c)1,Hemoglobin A1c (HbA1c)2,Hemoglobin A1c (HbA1c)3,BMI (kg/m^2)1,BMI (kg/m^2)2,BMI (kg/m^2)3
"(1000100164719, 2013-10-11 00:00:00)",0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(1000100164719, 2013-09-22 00:00:00)",1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(1000100164719, 2016-02-22 00:00:00)",0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
"(1000100164719, 2016-09-12 00:00:00)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
"(1000100164719, 2013-10-11 00:00:00)",0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(5050010047249, 2019-11-11 00:00:00)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"(11114000017506, 2006-07-25 00:00:00)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
"(11114000017506, 2006-02-21 00:00:00)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
"(11114000017506, 2008-03-25 00:00:00)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
for index, row in reducedDF.iterrows() :
    temp = int(index[0])
    featureVector = list(row)
    if temp not in VisitHistorySVD.keys():
        if temp in IDs :
            VisitHistorySVD[temp] = [1]
        else :
            VisitHistorySVD[temp] = [0]
    VisitHistorySVD[temp] = VisitHistorySVD[temp] + featureVector


# Fill in to 2500
for k,v in VisitHistorySVD.items() :
    fillLen = 2501 - len(v)
    VisitHistorySVD[k] = v + [0]*fillLen